In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from db import *
from psycopg2.errors import ForeignKeyViolation, UniqueViolation
from sklearn.pipeline import Pipeline
from sqlalchemy import delete, exc
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Session
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

/home/jovyan/work/db.py:8: SAWarning: Did not recognize type 'geometry' of column 'geom'
  mapping_hotel = Table(


In [2]:
hotels = pd.read_sql(
    sql=select(
        mapping_hotel.c.id,
        mapping_hotel.c.hotel_id,
        mapping_hotel.c.is_charter,
    ),
    con=engine.connect(),
).set_index("id")

rooms = pd.read_sql(
    sql=select(
        accommodation_hotel_room.c.id,
        accommodation_hotel_room.c.hotel_id,
        accommodation_hotel_room.c.name,
        mapping_hotel_room.c.room_code,
        mapping_hotel_room.c.room_type,
    ).join_from(mapping_hotel_room, accommodation_hotel_room),
    con=engine.connect(),
)

meals = pd.read_sql(
    sql=select(
        definitions_meal_plan.c.id.label("meal_id"),
        definitions_meal_plan.c.code,
    ),
    con=engine.connect(),
).set_index("code")

operators = pd.read_sql(
    sql=select(
        mapping_operator.c.operator_id,
        mapping_operator.c.external_id,
    ),
    con=engine.connect(),
).set_index("external_id")

In [3]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        ("data_encoder", BookingDataEncoder(hotels, rooms, meals, operators)),
    ]
)

In [4]:
for destination in ["OM"]:
    booking_data = BookingData(
        destination, "11/01/2023 00:00:00", "12/31/2026 00:00:00"
    )
    get_bookings = booking_data.get()

    if get_bookings is not None:
        df = pipeline.fit_transform(get_bookings)

In [5]:
with Session(engine) as session:
    stmt = delete(reservations_booking)
    session.execute(stmt)
    session.commit()


with Session(engine) as session:
    stmt = delete(reservations_booking_staging)
    session.execute(stmt)
    session.commit()

In [6]:
# Drop all rows that are not Orascom Hotels
orascom_hotels = [10001048, 10001106, 10001090, 10001020]
df = df[df["hotel_id"].isin(orascom_hotels)]

if df is not None:
    records = df.to_dict(orient="records")

    with Session(engine) as session:
        for record in records:
            try:
                stmt = insert(reservations_booking).values(record)
                session.execute(stmt)
                session.commit()
            except exc.IntegrityError as e:
                session.rollback()
                if isinstance(e.orig, UniqueViolation):
                    record["flag"] = "amendment"
                elif isinstance(e.orig, ForeignKeyViolation):
                    record["flag"] = "bad_mapping"
                    for field in ["hotel_id", "room_id", "meal_id", "operator_id"]:
                        record[field] = record[field] if record[field] != -1 else None

                stmt = (
                    insert(reservations_booking_staging)
                    .values(record)
                    .on_conflict_do_update(
                        constraint="reservations_booking_staging_ref_id_key",
                        set_=dict(stmt.excluded),
                    )
                )
                session.execute(stmt)
                session.commit()